In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.8.1+cu101', '4.6.1', '0.10.3')

In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
data_file = pd.read_csv("data/trofi.csv")

In [ ]:
data_file = data_file.rename(columns={'sentence': 'input_text'}) #abstract - sentence
data_file['target_text'] = data_file[['verb']] #title - metaphorical word

In [ ]:
papers = data_file[["input_text", "target_text"]]

In [ ]:
papers.head()

,input_text,target_text
0,An Energy Department spokesman says the sulfur...,absorb
1,The yellow beta carotene pigment absorbs blue ...,absorb
2,"This time , the ground absorbed the shock wave...",absorb
3,'' Vitamins could be passed right out of the b...,absorb
4,"As Eliot wrote : '' In a warm haze , the sultr...",absorb


In [ ]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel

eval_df = papers.sample(frac=0.1, random_state=42)
train_df = papers.drop(eval_df.index)

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 6,
    "num_train_epochs": 3,
}

# Create a Bart-base model
model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

CPU times: user 1.84 s, sys: 376 ms, total: 2.21 s
Wall time: 4.77 s


In [ ]:
%%time

# Train the model
model.train_model(train_df)

# Evaluate the model
result = model.eval_model(eval_df)
print(result)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/3363 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/561 [00:00<?, ?it/s]

...

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to outputs/.
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/374 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.24183904897273578}


{'eval_loss': 0.24183904897273578}
CPU times: user 7min 9s, sys: 3.83 s, total: 7min 13s
Wall time: 6min 45s


In [ ]:
for _ in range(65):

    random_idx = random.randint(0, len(eval_df)-1)

    abstract = eval_df.iloc[random_idx]['input_text']
    true_title = eval_df.iloc[random_idx]['target_text']

    # Predict with trained BART model
    predicted_title = model.predict([abstract])[0]

    print(f'True Title: {true_title}\n')
    print(f'Predicted Title: {predicted_title}\n')
    print(f'Abstract: {abstract}\n\n\n')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: miss

Predicted Title: miss

Abstract: Virginia Belden , who sells vacuum cleaners and sewing machines at the Towne East store , however , frets that some shoppers might miss the thrill of getting a bargain 





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: fill

Predicted Title: fill

Abstract: To fill these growing gaps , Congress increasingly has turned to employers to expand coverage to even more employees and non - employees 





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

True Title: plant

Predicted Title: plant

Abstract: Few things give him more satisfaction now , it seems , than the hundreds of roses , fruit trees and berry bushes the 57-year - old Mr. Lance and a grandson have planted together 





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: evaporate

Predicted Title: evaporate

Abstract: The low pressure causes liquid in the arm to evaporate and to be pulled into the pipe 





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: knock

Predicted Title: knock

Abstract: Seagram must decide by Feb. 9 whether to come up with a higher offer that would knock Grand Met out of the bidding war and frustrate its expansion strategy 



